# Parte 3: Recolección de información
### Web Scraping a "Semana" para investigación referente a los cambios climáticos en Colombia

### Paso 1: Importación de librerias

In [1]:
import pandas as pd
import time
import random
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support import expected_conditions as EC
from random import shuffle

### Paso 2: Definición de opciones para el navegador

In [2]:
chrome_options = Options()
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument("--start-maximized")
chrome_options.add_argument('--window-size=1420,1080')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument('--disable-gpu')
chrome_options.add_argument("--disable-notifications")
chrome_options.add_argument("--remote-debugging-port=9222")
chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
chrome_options.add_experimental_option('useAutomationExtension', False)
chrome_options.add_experimental_option("excludeSwitches", ["disable-popup-blocking"])

### Paso 3: Definición de clase personalizada

In [3]:
# Clase para personalización
class bcolors:
    WARNING = '\033[93m'
    FAIL = '\033[91m'

# Control de errores
try:   
    # Definición de función
    def Seconds(result):
        hours = round(result/3600)
        minutes = round((result - (hours * 3600))/60)
        seconds = round(result - ((hours * 3600) + (minutes * 60)))
        if seconds < 0: seconds = seconds*-1
        if minutes < 0: minutes = minutes*-1
        final_time = "El sitema tardó: " + str(hours) + " horas " + str(minutes) + " minutos " + str(seconds) + " segundos en finalizar su ejeción"
        return final_time
    
except Exception as e: 
    print(bcolors.FAIL + "Revisa el error => ", e) # Impresión del error personalizado

### Paso 4: Lectura de archivo de noticias

In [4]:
# Definición de la clase
class IterableClass:
    
    # Función inicial o constructor
    def __init__(self, list_values = None): # None es para que sea opcional
        self.list_values = list_values
        self.dateframe = pd.read_csv(r'df_clean_5.csv') # lectura del archivo
        self.i = 0
        
    # Retornar un iterador
    def __iter__(self):
        return self
    
    # Retorna un elemento del contenedor
    def __next__(self):
        if self.i >= len(self.list_values):
            print("\nNo hay más valores a recorrer... ⬇")
            raise StopIteration
            
        item = self.list_values[self.i]
        self.i += 1
        return item  

### Paso 5: Instacia del objeto e impresión del DataFrame

In [5]:
# Instancia del objeto
IC = IterableClass()

# 5.1. Eliminación de columna que se creo en por error "Unnamed: 0" .drop('Unnamed: 0', axis=1)
# 5.2. Reseteo de index por la acción de combinación de DataFrames .reset_index().drop(['index'], axis=1)
IC.dateframe.drop('Unnamed: 0', axis=1).reset_index().drop(['index'], axis=1)

,headers_text,headers_link
0,Huila dice no a hidroeléctricas que atenten co...,https://www.semana.com/el-quimbo-huila-dice-no...
1,¿Cuáles fueron los grandes hechos ambientales ...,https://www.semana.com/cuales-fueron-los-grand...
2,Un año después de la tragedia minera más grand...,https://www.semana.com/tragedia-de-samarco-y-r...
3,¿En qué va la política ambiental de Duque?\r\n...,https://www.semana.com/medio-ambiente/articulo...
4,La paradoja de la paz: cuatro historias de des...,https://www.semana.com/medio-ambiente/articulo...
...,...,...
295,Proyectan reverdecer a Cúcuta con la siembra 1...,https://www.semana.com/medio-ambiente/articulo...
296,Se inició la restauración del parque Entrenube...,https://www.semana.com/parque-entrenubes-comie...
297,"Un ejército de jóvenes que siembren árboles, o...",https://www.semana.com/medio-ambiente/articulo...
298,Pretenden sembrar 5 millones de árboles en dos...,https://www.semana.com/pretenden-sembrar-5-mil...


### Paso 6: Definición de función para unificar los diferentes parrafos de la noticia

In [6]:
def JoinParagraphs(paragraphs):
    TextParagraphs = ""
    for paragraph in paragraphs:
        TextParagraphs = TextParagraphs + paragraph.text
        
    return TextParagraphs

### Paso 7: Definición de función para unificar los diferentes temas de la noticia

In [7]:
def JoinThemes(themes):
    TextThemes = ""
    for theme in themes:
        TextThemes = TextThemes + theme.text + ", "
    return TextThemes

### Paso 8: Recolección de información para cada noticia

In [8]:
# 8.1 Declaraciones de variables
titles = []
summaries = []
authors = []
dates = []
paragraphs = []
themes = []

In [9]:
# PASO 8.2: Aviso para el usuario final de que se inicio el web-scraping --------------------------------------------------------------#
print("Se inició el Web Scraping a 'Semana', por favor espere...") # Mensaje para visualización del inicio del proceso

# 8.2.1 Variable para definir el inicio de tiempo en ejecución
start = time.time()

Se inició el Web Scraping a 'Semana', por favor espere...


In [10]:
# PASO 8.3 Estructura base para acceder a la página en cuestión ----------------------------------------------------------------------#
service = Service(executable_path=r'../../chromedriver-win64/chromedriver.exe') # Creación del servicio mediante el driver
options = webdriver.ChromeOptions() # Definición de variable con las opciones para chrome
driver = webdriver.Chrome(service=service, options=options) # Creación del objeto del WebDriver

# Asignación de dataframe a una variable
links = IC.dateframe

# Control del error al momento de finalizar el páginado
try:
    
    # Iteración para acceso a los links del dataframe
    for i in range(len(links)):
        
        news = {
            "titles" : [],
            "summaries" : [],
            "authors" : [],
            "dates" : [],
            "paragraphs" : [],
            "themes" : [],
        }
        
        # Acceso a cada link del dataframe
        driver.get(links.iloc[i]['headers_link'])

        # Títulos
        # ****************************************************************************************************************
        title = driver.find_elements(By.XPATH, '//*[@id="fusion-app"]/div/main/article/div/div[2]/h1')
        if title:
            for i in title:
                print(titles.append(i.text))
        else: titles.append('...')
        # ****************************************************************************************************************

        # Resúmenes
        # ****************************************************************************************************************
        summary = driver.find_elements(By.XPATH, '//*[@id="fusion-app"]/div/main/article/div/div[2]/p')
        if summary: 
            for i in summary:
                print(summaries.append(i.text))
        else: summaries.append('...')
        # ****************************************************************************************************************

        # Autor
        # ****************************************************************************************************************
        date_a = driver.find_elements(By.XPATH, '//*[@id="fusion-app"]/div/main/article/div/div[2]/div[2]/div')
        if date_a: 
            if len(date_a) == 1:
                author_a = driver.find_elements(By.XPATH, '//*[@id="fusion-app"]/div/main/article/div/div[2]/div[2]/a/div')
                if author_a:
                    for i in author_a:
                        print(authors.append(i.text))
                else: authors.append('...')
            elif len(date_a) > 1:
                author_b = driver.find_elements(By.XPATH, '//*[@id="fusion-app"]/div/main/article/div/div[2]/div[2]/div[1]')
                if author_b:
                    for i in author_b:
                        print(authors.append(i.text))
                else: authors.append('...')
            else: authors.append('...')
        else: authors.append('...')
        # Test
        author_a = None
        author_b = None
        date_a = None
        # ****************************************************************************************************************

        # Fecha
        # ****************************************************************************************************************
        date_a = driver.find_elements(By.XPATH, '//*[@id="fusion-app"]/div/main/article/div/div[2]/div[2]/div')
        if date_a: 
            if len(date_a) == 1:
                if date_a:
                    for i in date_a:
                        print(dates.append(i.text))
                else: dates.append('...')
            elif len(date_a) > 1:
                date_b = driver.find_elements(By.XPATH, '//*[@id="fusion-app"]/div/main/article/div/div[2]/div[2]/div[2]')
                if date_b:
                    for i in date_b:
                        print(dates.append(i.text))
                else: dates.append('...')
            else: dates.append('...')
        else: dates.append('...')
        # Test
        author_a = None
        date_a = None
        date_b = None
        # ****************************************************************************************************************
        
        # Parrafos
        # ****************************************************************************************************************
        paragraphs_a = driver.find_elements(By.CLASS_NAME, 'prose')
        if paragraphs_a: 
            if len(paragraphs_a) > 1:
                paragraphs.append(JoinParagraphs(paragraphs_a))
            else:
                paragraphs.append(paragraphs_a[0].text)
        else: paragraphs.append('...')
        # ****************************************************************************************************************

        # Temas
        # ****************************************************************************************************************
        tags = driver.find_elements(By.CSS_SELECTOR, 'a.inline-block')
        if tags: 
            if len(tags) > 1:
                themes.append(JoinThemes(tags))
            else:
                themes.append(tags[0].text)
        else: themes.append('...')
        # ****************************************************************************************************************
                
        # Construcción del diccionario 
        news = {
            "titles" : titles,
            "summaries" : summaries,
            "authors" : authors,
            "dates" : dates,
            "paragraphs" : paragraphs,
            "themes" : themes
        }

        # Construcción inicial del dataframe
        df = pd.DataFrame(news)

        # (Opcional) Modo DEBUG, sirve para validar como el DataFrame se está llenando según las iteraciones
        display(df)
    else:
        driver.quit();
        
except Exception as e:
    display("Exception :", e)
    driver.quit() 
except NoSuchElementException as nse:
    display("NoSuchElementException :", nse)
    driver.quit()
except TimeoutException as tim:
    display("TimeoutException :", tim)
    driver.quit()

None
None
None


,titles,summaries,authors,dates,paragraphs,themes
0,Huila dice no a hidroeléctricas que atenten co...,El 11 de noviembre en una audiencia pública na...,...,2 de noviembre de 2016,La hidroeléctrica de El Quimbo entró en funcio...,"El Quimbo, Huila, Energía eléctrica, EMGESA, M..."


None
None
None


,titles,summaries,authors,dates,paragraphs,themes
0,Huila dice no a hidroeléctricas que atenten co...,El 11 de noviembre en una audiencia pública na...,...,2 de noviembre de 2016,La hidroeléctrica de El Quimbo entró en funcio...,"El Quimbo, Huila, Energía eléctrica, EMGESA, M..."
1,¿Cuáles fueron los grandes hechos ambientales ...,"Deforestación, contaminación de los mares, des...",...,15 de diciembre de 2017,Los temas ambientales cobran cada vez mayor im...,"Deforestación, Consultas populares, Plástico, ..."


None
None
None


...

,titles,summaries,authors,dates,paragraphs,themes
0,Huila dice no a hidroeléctricas que atenten co...,El 11 de noviembre en una audiencia pública na...,...,2 de noviembre de 2016,La hidroeléctrica de El Quimbo entró en funcio...,"El Quimbo, Huila, Energía eléctrica, EMGESA, M..."
1,¿Cuáles fueron los grandes hechos ambientales ...,"Deforestación, contaminación de los mares, des...",...,15 de diciembre de 2017,Los temas ambientales cobran cada vez mayor im...,"Deforestación, Consultas populares, Plástico, ..."
2,Un año después de la tragedia minera más grand...,"Las empresas BHP, la mayor minera del mundo, y...",...,2 de noviembre de 2016,En un valle arrasado por el alud de barro que ...,"Río Doce, Minería, Brasil,"
3,¿En qué va la política ambiental de Duque?,Un año después de comenzar el mandato del pres...,...,6 de agosto de 2019,"Este lunes, el ministro de Ambiente y Desarrol...","Iván Duque, presidente de Colombia, Ministro d..."
4,La paradoja de la paz: cuatro historias de des...,En muchos de los territorios donde antes domin...,...,26 de junio de 2018,El país perdió casi 220.000 hectáreas de bosqu...,"Deforestación, Paz, Farc, Caquetá, Tumaco, Gua..."
...,...,...,...,...,...,...
293,"SOS para salvar un árbol de 118 años, símbolo ...","En el corregimiento de Robles, en el Valle, la...",...,12 de agosto de 2021,Árboles\nJamundí\nValle del Cauca\nUnirme al c...,"Árboles, Jamundí, Valle del Cauca,"
294,"1,5 millones de árboles se han sembrado en el ...",Un guayacán se convirtió en el árbol 1.500.000...,...,24 de diciembre de 2020,Valle del Cauca\nSiembra de árboles en el Vall...,"Valle del Cauca, Siembra de árboles en el Vall..."
295,Proyectan reverdecer a Cúcuta con la siembra 1...,La iniciativa apunta al embellecimiento paisaj...,...,27 de noviembre de 2020,Arborización en Cúcuta\nSiembra de árboles en ...,"Arborización en Cúcuta, Siembra de árboles en ..."
296,Se inició la restauración del parque Entrenube...,En la jornada participaron comunidades y colec...,...,13 de junio de 2020,Quemas y tala en parque Entrenubes de Bogotá\n...,"Quemas y tala en parque Entrenubes de Bogotá, ..."


None
None
None


,titles,summaries,authors,dates,paragraphs,themes
0,Huila dice no a hidroeléctricas que atenten co...,El 11 de noviembre en una audiencia pública na...,...,2 de noviembre de 2016,La hidroeléctrica de El Quimbo entró en funcio...,"El Quimbo, Huila, Energía eléctrica, EMGESA, M..."
1,¿Cuáles fueron los grandes hechos ambientales ...,"Deforestación, contaminación de los mares, des...",...,15 de diciembre de 2017,Los temas ambientales cobran cada vez mayor im...,"Deforestación, Consultas populares, Plástico, ..."
2,Un año después de la tragedia minera más grand...,"Las empresas BHP, la mayor minera del mundo, y...",...,2 de noviembre de 2016,En un valle arrasado por el alud de barro que ...,"Río Doce, Minería, Brasil,"
3,¿En qué va la política ambiental de Duque?,Un año después de comenzar el mandato del pres...,...,6 de agosto de 2019,"Este lunes, el ministro de Ambiente y Desarrol...","Iván Duque, presidente de Colombia, Ministro d..."
4,La paradoja de la paz: cuatro historias de des...,En muchos de los territorios donde antes domin...,...,26 de junio de 2018,El país perdió casi 220.000 hectáreas de bosqu...,"Deforestación, Paz, Farc, Caquetá, Tumaco, Gua..."
...,...,...,...,...,...,...
294,"1,5 millones de árboles se han sembrado en el ...",Un guayacán se convirtió en el árbol 1.500.000...,...,24 de diciembre de 2020,Valle del Cauca\nSiembra de árboles en el Vall...,"Valle del Cauca, Siembra de árboles en el Vall..."
295,Proyectan reverdecer a Cúcuta con la siembra 1...,La iniciativa apunta al embellecimiento paisaj...,...,27 de noviembre de 2020,Arborización en Cúcuta\nSiembra de árboles en ...,"Arborización en Cúcuta, Siembra de árboles en ..."
296,Se inició la restauración del parque Entrenube...,En la jornada participaron comunidades y colec...,...,13 de junio de 2020,Quemas y tala en parque Entrenubes de Bogotá\n...,"Quemas y tala en parque Entrenubes de Bogotá, ..."
297,"Un ejército de jóvenes que siembren árboles, o...","En una columna para The New York Times, Collin...",...,2 de junio de 2020,Sembratón de árboles\nUnirme al canal de Whats...,Sembratón de árboles


None
None
None


,titles,summaries,authors,dates,paragraphs,themes
0,Huila dice no a hidroeléctricas que atenten co...,El 11 de noviembre en una audiencia pública na...,...,2 de noviembre de 2016,La hidroeléctrica de El Quimbo entró en funcio...,"El Quimbo, Huila, Energía eléctrica, EMGESA, M..."
1,¿Cuáles fueron los grandes hechos ambientales ...,"Deforestación, contaminación de los mares, des...",...,15 de diciembre de 2017,Los temas ambientales cobran cada vez mayor im...,"Deforestación, Consultas populares, Plástico, ..."
2,Un año después de la tragedia minera más grand...,"Las empresas BHP, la mayor minera del mundo, y...",...,2 de noviembre de 2016,En un valle arrasado por el alud de barro que ...,"Río Doce, Minería, Brasil,"
3,¿En qué va la política ambiental de Duque?,Un año después de comenzar el mandato del pres...,...,6 de agosto de 2019,"Este lunes, el ministro de Ambiente y Desarrol...","Iván Duque, presidente de Colombia, Ministro d..."
4,La paradoja de la paz: cuatro historias de des...,En muchos de los territorios donde antes domin...,...,26 de junio de 2018,El país perdió casi 220.000 hectáreas de bosqu...,"Deforestación, Paz, Farc, Caquetá, Tumaco, Gua..."
...,...,...,...,...,...,...
295,Proyectan reverdecer a Cúcuta con la siembra 1...,La iniciativa apunta al embellecimiento paisaj...,...,27 de noviembre de 2020,Arborización en Cúcuta\nSiembra de árboles en ...,"Arborización en Cúcuta, Siembra de árboles en ..."
296,Se inició la restauración del parque Entrenube...,En la jornada participaron comunidades y colec...,...,13 de junio de 2020,Quemas y tala en parque Entrenubes de Bogotá\n...,"Quemas y tala en parque Entrenubes de Bogotá, ..."
297,"Un ejército de jóvenes que siembren árboles, o...","En una columna para The New York Times, Collin...",...,2 de junio de 2020,Sembratón de árboles\nUnirme al canal de Whats...,Sembratón de árboles
298,Pretenden sembrar 5 millones de árboles en dos...,En 11 departamentos se realizará la primera gr...,...,28 de enero de 2020,Siembra de árboles en Colombia\nPreservación d...,"Siembra de árboles en Colombia, Preservación d..."


In [11]:
df_news_clean = df.drop_duplicates(subset=["titles"])

In [12]:
df_news_clean.to_csv("df_news/df_news_clean_5.csv")

In [13]:
# Variable para definir el final de tiempo en ejecución
end = time.time()

# Variable para calcular tiempo y para enviar a la función
result = end-start

#Impresión del tiempo que tardó
print(Seconds(result))

El sitema tardó: 0 horas 21 minutos 29 segundos en finalizar su ejeción


In [14]:
df_news_clean

,titles,summaries,authors,dates,paragraphs,themes
0,Huila dice no a hidroeléctricas que atenten co...,El 11 de noviembre en una audiencia pública na...,...,2 de noviembre de 2016,La hidroeléctrica de El Quimbo entró en funcio...,"El Quimbo, Huila, Energía eléctrica, EMGESA, M..."
1,¿Cuáles fueron los grandes hechos ambientales ...,"Deforestación, contaminación de los mares, des...",...,15 de diciembre de 2017,Los temas ambientales cobran cada vez mayor im...,"Deforestación, Consultas populares, Plástico, ..."
2,Un año después de la tragedia minera más grand...,"Las empresas BHP, la mayor minera del mundo, y...",...,2 de noviembre de 2016,En un valle arrasado por el alud de barro que ...,"Río Doce, Minería, Brasil,"
3,¿En qué va la política ambiental de Duque?,Un año después de comenzar el mandato del pres...,...,6 de agosto de 2019,"Este lunes, el ministro de Ambiente y Desarrol...","Iván Duque, presidente de Colombia, Ministro d..."
4,La paradoja de la paz: cuatro historias de des...,En muchos de los territorios donde antes domin...,...,26 de junio de 2018,El país perdió casi 220.000 hectáreas de bosqu...,"Deforestación, Paz, Farc, Caquetá, Tumaco, Gua..."
...,...,...,...,...,...,...
295,Proyectan reverdecer a Cúcuta con la siembra 1...,La iniciativa apunta al embellecimiento paisaj...,...,27 de noviembre de 2020,Arborización en Cúcuta\nSiembra de árboles en ...,"Arborización en Cúcuta, Siembra de árboles en ..."
296,Se inició la restauración del parque Entrenube...,En la jornada participaron comunidades y colec...,...,13 de junio de 2020,Quemas y tala en parque Entrenubes de Bogotá\n...,"Quemas y tala en parque Entrenubes de Bogotá, ..."
297,"Un ejército de jóvenes que siembren árboles, o...","En una columna para The New York Times, Collin...",...,2 de junio de 2020,Sembratón de árboles\nUnirme al canal de Whats...,Sembratón de árboles
298,Pretenden sembrar 5 millones de árboles en dos...,En 11 departamentos se realizará la primera gr...,...,28 de enero de 2020,Siembra de árboles en Colombia\nPreservación d...,"Siembra de árboles en Colombia, Preservación d..."
